In [1]:
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense
import torch
import os
import cv2
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split


In [3]:
os.chdir("C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\networks")

In [47]:
cmap_colors = [(0.00, 0.0, 0.0), (1.0, 1.0, 1.0)] # White to Grey
cmap_name = 'custom_colormap'
cmapGrey = LinearSegmentedColormap.from_list(cmap_name, cmap_colors)

In [48]:
def read_file(date):
    mapDf = pd.read_csv('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\SSMR\\iceDataframes\\maps\\' + date + 'map.csv')
    mapDf = mapDf.drop(['Unnamed: 0'], axis=1)
    outlineDf = pd.read_csv('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\SSMR\\iceDataframes\\oceans\\' + date + 'oceans.csv')
    outlineDf = outlineDf.drop(['Unnamed: 0'], axis=1)
    return mapDf, outlineDf

In [49]:
def prep_images():
    os.chdir('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\SSMR\\iceDataframes\\oceans')
    fnames = os.listdir()
    dates = [file[:8] for file in fnames]
    i=0
    for date in dates:
        try:
            mapDf, outlineDf = read_file(date)
            os.chdir('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\networks\\CNNimages')
            fig, ax = plt.subplots(figsize=(304/77, 448/77))
            im1 = ax.imshow(mapDf, cmap='jet_r')
            im2 = ax.imshow(outlineDf, cmap=cmapGrey, alpha=1)
            plt.axis('off')
            plt.savefig(f'{date}_CNN.png', bbox_inches='tight', pad_inches=0)
            plt.close()
        except:
            print(f'Failed: {date}')
        i+=1
        if (i%100) == 0:
            print(f'Done with: {i}')

In [50]:
def extract_images():
    imgsPaths = []
    dates = []
    os.chdir('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\networks\\CNNimages')
    fnames = os.listdir()
    i=0
    print(len(fnames))
    for file in fnames:
        try:
            dates.append(pd.to_datetime(file[4:8], format='%m%d'))
            imgsPaths.append(f'C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\networks\\CNNimages\\{file}')
            if (i%100) == 0:
                print(f'Done with: {i}')
            i+=1
        except:
            pass
    df = pd.DataFrame({'Date': dates, 'ImagePath': imgsPaths})
    df['DayOfYearSin'] = np.sin(2*np.pi*df['Date'].dt.dayofyear/365)
    df['DayOfYearCos'] = np.cos(2*np.pi*df['Date'].dt.dayofyear/365)
    return df

In [71]:
def preprocess_image(path):
    img = cv2.imread(path)
    img = img / 255.0
    return img

In [73]:
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img = np.array(img)
        images.append(img)
    return np.array(images)/255

In [ ]:
df = extract_images()

In [63]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(448, 304, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2),
])

In [87]:
def train_model(model):
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
    X_train, X_test, y_train, y_test = train_test_split(df['ImagePath'].values,
                                                        df[['DayOfYearSin', 'DayOfYearCos']].values,
                                                        test_size=0.2, random_state=42)
    X_train = load_images(X_train)
    X_test = load_images(X_test)

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)
    return history

In [84]:
df = extract_images()

1507
Done with: 0
Done with: 100
Done with: 200
Done with: 300
Done with: 400
Done with: 500
Done with: 600
Done with: 700
Done with: 800
Done with: 900
Done with: 1000
Done with: 1100
Done with: 1200
Done with: 1300
Done with: 1400


In [85]:
train_model(model)

Epoch 1/10
37/37 [==============================] - 49s 1s/step - loss: -644596736.0000 - accuracy: 0.5973 - val_loss: -720168704.0000 - val_accuracy: 0.5445
Epoch 2/10
37/37 [==============================] - 45s 1s/step - loss: -773842560.0000 - accuracy: 0.6041 - val_loss: -866834624.0000 - val_accuracy: 0.5342
Epoch 3/10
37/37 [==============================] - 45s 1s/step - loss: -931347840.0000 - accuracy: 0.5981 - val_loss: -1042969984.0000 - val_accuracy: 0.5514
Epoch 4/10
37/37 [==============================] - 49s 1s/step - loss: -1118358144.0000 - accuracy: 0.6084 - val_loss: -1262646400.0000 - val_accuracy: 0.5548
Epoch 5/10
37/37 [==============================] - 64s 2s/step - loss: -1351138944.0000 - accuracy: 0.5973 - val_loss: -1527923968.0000 - val_accuracy: 0.5616
Epoch 6/10
37/37 [==============================] - 81s 2s/step - loss: -1638620544.0000 - accuracy: 0.5964 - val_loss: -1847288320.0000 - val_accuracy: 0.5616
Epoch 7/10
37/37 [===========================

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense
import torch
import os
import cv2
from matplotlib.colors import LinearSegmentedColormap
os.chdir("C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\networks")
cmap_colors = [(0.00, 0.0, 0.0), (1.0, 1.0, 1.0)] # White to Grey
cmap_name = 'custom_colormap'
cmapGrey = LinearSegmentedColormap.from_list(cmap_name, cmap_colors)
def read_file(date):
    mapDf = pd.read_csv('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\SSMR\\iceDataframes\\maps\\' + date + 'map.csv')
    mapDf = mapDf.drop(['Unnamed: 0'], axis=1)
    outlineDf = pd.read_csv('C:\\Users\\jleus\\PycharmProjects\\APLResearch\\Ice Data\\SSMR\\iceDataframes\\oceans\\' + date + 'oceans.csv')
    outlineDf = outlineDf.drop(['Unnamed: 0'], axis=1)
    return mapDf, outlineDf
def prep_imag